In [ ]:
import os

from typing import List
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv

load_dotenv()

True

In [15]:
def read_file_and_create_chunks(file_path: str) -> str:
    """
    Lê o conteúdo de um arquivo PDF ou CSV e divide o texto em chunks (partes menores) com sobreposição para preservar o contexto
    
    Args:
        file_path: Caminho para o arquivo PDF ou CSV.
    
    Returns:
        List[Document]: Lista de chunks representados como documentos.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=500,
        length_function=len,
    )

    file_extension = file_path.split(".")[-1]

    if file_extension == "pdf":
        loader = PyPDFLoader(file_path)
    elif file_extension == "csv":
        loader = CSVLoader(file_path)
    else:
        raise ValueError(f"Tipo de loader não suportado: {file_extension}. Use 'pdf' ou 'csv'.")

    return loader.load_and_split(text_splitter=text_splitter)



In [16]:
def create_vector_store(chunks: List[Document], openai_api_key: str) -> Chroma:
    """
    Cria um banco de vetores para armazenar os embeddings gerados a partir dos chunks.
    
    Args:
        chunks: Lista de chunks do texto.
        openai_api_key: Chave da API da OpenAI para geração de embeddings.
    
    Returns:
        Chroma: Banco de dados vetorial criado.
    """
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    db = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="db")
    print("Banco de vetores criado com sucesso.\n")
    return db

In [17]:
def setup_prompt() -> PromptTemplate:
    """
    Configura o prompt para geração de respostas baseadas no contexto recuperado.
    
    Returns:
        PromptTemplate: Template de prompt configurado.
    """
    prompt = PromptTemplate.from_template("""
    Answer the user query based on context. If you don't know the answer 
    or the context does not have the answer, say that you don't know.
    ALWAYS answer in pt-BR. 
    Provide the source of your knowledge. I want the answer with a link or document reference.

    ## CONTEXT
    {contexto}

    ## USER QUERY
    {pergunta}
    """)
    return prompt

In [18]:
def rag_pipeline(db: Chroma, user_query: str, openai_api_key: str) -> str:
    """
    Pipeline de RAG que recupera informações relevantes e gera uma resposta.
    
    Args:
        db: Banco de vetores criado.
        user_query: Pergunta do usuário.
        openai_api_key: Chave da API da OpenAI para geração de respostas.
    
    Returns:
        str: Resposta gerada com base no contexto recuperado.
    """
    print("Buscando informações relevantes no banco de vetores...")
    context = db.similarity_search_with_relevance_scores(user_query, k=3)
    context = [c for c, score in context if score >= 0.7]

    if not context:
        return "Eu não sou capaz de responder a essa pergunta."

    context_text = "\n\n".join([f"## Documento {i+1}\n{chunk.page_content}" for i, chunk in enumerate(context)])
    
    print("Gerando resposta com base no contexto...\n")
    prompt = setup_prompt()
    chain = (prompt |
             ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=openai_api_key) |
             StrOutputParser())
    
    print("Resposta:\n")
    
    return chain.invoke({"contexto": context_text, "pergunta": user_query})

In [20]:

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

file_path = "./imoveis.csv"

chunks = read_file_and_create_chunks(file_path)

db = create_vector_store(chunks, OPENAI_API_KEY)

user_query = "Qual imóvel tem vista para praia?"

resposta = rag_pipeline(db, user_query, OPENAI_API_KEY)
    
print(resposta)


Banco de vetores criado com sucesso.

Buscando informações relevantes no banco de vetores...
Gerando resposta com base no contexto...

Resposta:

O imóvel que tem vista para a praia é o apartamento localizado na Av. Praia de Ponta Negra, 1001. Ele possui 2 quartos, 2 banheiros, 1 vaga e está à venda por R$ 850.000. A descrição menciona que tem vista para o mar, sacada e acabamento premium.

Fonte: Documento 1.
